In [20]:
# import modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import necessary module(s) for random forest
from sklearn.ensemble import RandomForestRegressor
#import necessary module(s) for LASSO
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#import necessary module(s) for RIDGE
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
#import necessary module(s) for LightGBM
import lightgbm as lgb
#import necessary module(s) for XGBoost
import xgboost as xgb

# files
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


## import train- and test datasets

In [21]:
#import training dataset
df_train = pd.read_csv(r'C:/Users/foeke/OneDrive/Documenten/submitting paper/All scripts - paper/data/GlobalModelData/PredictingDataset.csv', sep=',')
df_train

,Unnamed: 0,Longitude,Latitude,mean_value_NO2,nightlight_450,nightlight_3150,population_1000,population_3000,road_class_2_25,road_class_3_3000,road_class_3_300,trop_mean_filt_2019,BldDen100,NDVI,trafBuf25,trafBuf50
0,0,10.008293,48.397079,29.250500,22.585377,13.402575,8123.319336,54316.64453,0.000000,203502.42190,4167.064941,0.000058,1.000663,2310.0,0.000000,0.000000
1,1,10.021227,53.523262,32.098936,23.546194,22.862799,9558.207031,76786.28125,0.000000,99596.04688,1850.278076,0.000081,0.978245,2518.0,0.000000,0.000000
2,2,10.053859,53.592301,58.703744,18.290140,16.493486,10471.541020,88439.60938,46.359188,265563.71880,2016.838379,0.000076,0.914638,3216.0,674.791667,674.791667
3,3,10.081685,53.529020,24.249276,23.645823,18.248226,11165.945310,76961.00000,0.000000,97921.56250,478.363281,0.000074,0.712983,1052.0,0.000000,0.000000
4,4,10.096292,48.847917,19.912116,14.026642,5.514785,6017.511230,30814.58984,0.000000,176265.68750,3338.221191,0.000047,1.000641,3240.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,477,9.979031,48.396836,27.221808,29.130899,12.865678,10829.854490,54732.38281,0.000000,198280.70310,3127.346436,0.000058,0.964609,2607.0,0.000000,0.000000
478,478,9.981599,53.529158,32.763475,33.719387,36.813637,9855.304688,83610.52344,0.000000,117941.50000,1480.759766,0.000088,1.000696,NaN,0.000000,0.000000
479,479,9.990569,53.507930,24.966811,17.919800,25.516184,9541.226562,76877.67188,0.000000,94117.64063,2670.007568,0.000082,0.485096,4338.0,0.000000,0.000000
480,480,9.997256,53.681993,39.256824,13.382681,7.228415,7302.048828,44593.98438,54.294807,171093.43750,2462.917969,0.000063,1.000694,4006.0,674.791667,674.791667


In [22]:
#replace NA with 0
df_train=df_train.fillna(0)
#remove unnecessary information in training dataset
df_train = df_train.drop(['Unnamed: 0', 'Longitude', 'Latitude'], axis=1)

#import testing dataset - to this unprocessed dataset, the predicted NO2 will eventually be added.

#AMSTERDAM
#df_test_xy = pd.read_csv(r'C:/Users/foeke/OneDrive/Documenten/submitting paper/TooBigData/SpatialPredictionPatterns/grid100_GlobalPredictors-Amsterdam.csv', sep=',')

# #HAMBURG
#df_test_xy = pd.read_csv(r'C:/Users/foeke/OneDrive/Documenten/submitting paper/TooBigData/SpatialPredictionPatterns/grid100_GlobalPredictors-HH.csv', sep=',')

# #Bayreuth
#df_test_xy = pd.read_csv(r'C:/Users/foeke/OneDrive/Documenten/submitting paper/TooBigData/SpatialPredictionPatterns/grid100_GlobalPredictors-Bayreuth.csv', sep=',')

# # #Utrecht
df_test_xy = pd.read_csv(r'C:/Users/foeke/OneDrive/Documenten/submitting paper/TooBigData/SpatialPredictionPatterns/grid100_GlobalPredictors-Utrecht.csv', sep=',')

#remove unnecessary information in testing dataset
df_test_processed = df_test_xy.drop(['Unnamed: 0', 'Longitude', 'Latitude'], axis=1)
df_test_processed = df_test_processed.rename(columns={'trop_mean_filt': 'trop_mean_filt_2019'})

#define response- and predictor variables
Y_train = df_train['mean_value_NO2']
X_train = df_train.drop(['mean_value_NO2'], axis=1)
X_test = df_test_processed

#validate if columns of training and testing dataset are the same
print(len(X_test.columns))
print(len(X_train.columns))

12
12


In [23]:
# specify city so that it is included in output-name.
city = "Utrecht100"

## model 1 - Random Forest

In [24]:
## PARAMETERS FOR MODEL - RANDOM FOREST

# # Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, min_samples_split=10,
                          min_samples_leaf=5,max_features=4,max_depth=10, bootstrap=True )
# Train the model on training data
rf.fit(X_train, Y_train)

#predict NO2 values, based on random forest model that is trained by the training dataset
predicted_NO2 = rf.predict(X_test)

#add predicted values as a column to copy of original dataset ("df_test_xy")
df_test_xy_RF = df_test_xy
df_test_xy_RF['predicted_NO2_RF'] = predicted_NO2

#export to .xlsx & .csv
df_test_xy_RF.to_excel('Predicting NO2-RF'+ str(city)+'_xy.xlsx')
df_test_xy_RF.to_csv('Predicting NO2-RF'+ str(city)+'_xy.csv')

## model 2 - LASSO

In [25]:
#set alpha to 0.1 and train model based on training data
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, Y_train)

#predict NO2 values, based on LASSO model that is trained by the training dataset
predicted_NO2= model_lasso.predict(X_test)

#add predicted values as a column to copy of the original dataset ("df_test_xy")
df_test_xy_LAS = df_test_xy
df_test_xy_LAS['predicted_NO2_LASSO'] = predicted_NO2

#export to .xlsx & .csv
df_test_xy_LAS.to_excel('Predicting NO2-RF-LAS-'+ str(city)+'_xy.xlsx')
df_test_xy_LAS.to_csv('Predicting NO2-RF-LAS-'+ str(city)+'_xy.csv')

## model 3 - RIDGE

In [26]:
#set alpha to 0.1 and train model based on training data
model_ridge = Ridge(alpha = 0.1)
model_ridge.fit(X_train, Y_train) #again, data is standardized

#predict NO2 values, based on RIDGE model that is trained by the training dataset
predicted_NO2= model_ridge.predict(X_test)

#add predicted values as a column to copy of the original dataset ("df_test_xy")
df_test_xy_RID = df_test_xy
df_test_xy_RID['predicted_NO2_RIDGE'] = predicted_NO2

#export to .xlsx & .csv
df_test_xy_RID.to_excel('Predicting NO2-RF-LAS-RID-'+ str(city)+'_xy.xlsx')
df_test_xy_RID.to_csv('Predicting NO2-RF-LAS-RID-'+ str(city)+'_xy.csv')

## model 4 - LightGBM

In [27]:
#set model parameters and train model on training dataset
model = lgb.LGBMRegressor(reg_alpha =2, reg_lambda = 0, max_depth = 5, learning_rate = 0.0005, n_estimators =50000, random_state=42)
model.fit(X_train, Y_train)

#predict NO2 values, based on LightGBM model that is trained by the training dataset
predicted_NO2 = model.predict(X_test)

#add predicted values as a column to copy of the original dataset ("df_test_xy")
df_test_xy_LGB = df_test_xy
df_test_xy_LGB['predicted_NO2_LightGBM'] = predicted_NO2

#export to .xlsx & .csv
df_test_xy_LGB.to_excel('Predicting NO2-RF-LAS-RID-LGB-'+ str(city)+'_xy.xlsx')
df_test_xy_LGB.to_csv('Predicting NO2-RF-LAS-RID-LGB-'+ str(city)+'_xy.csv')

## model 5A - XGBoost

In [28]:
#set model parameters and train model on training dataset
xg_reg = xgb.XGBRegressor(gamma = 5,  reg_alpha =2, reg_lambda=0, max_depth = 5, learning_rate = 0.0005, n_estimators=50000, random_state=42)
xg_reg.fit(X_train, Y_train)

#predict NO2 values, based on XGBoost model that is trained by the training dataset
predicted_NO2 = xg_reg.predict(X_test)

#add predicted values as a column to copy of the original dataset ("df_test_xy")
df_test_xy_XGB = df_test_xy
df_test_xy_XGB['predicted_NO2_XGBoost'] = predicted_NO2

#export to .xlsx & .csv
df_test_xy_XGB.to_excel('C:/Users/foeke/OneDrive/Documenten/submitting paper/TooBigData/SpatialPredictionPatterns/Predicting NO2-AllModels'+ str(city)+'_xy.xlsx')
df_test_xy_XGB.to_csv('C:/Users/foeke/OneDrive/Documenten/submitting paper/TooBigData/SpatialPredictionPatterns/Predicting NO2-AllModels'+ str(city)+'_xy.csv')